# iOCT then OCT

In this notebook, I first train a pix2pix model on the iOCT domain. Then I finetune it on the OCT domain to gain better quality layers.

TODOs:
- Get a unified-labeled iOCT dataset
- Get a unified-labeled OCT dataset
- (Optional) Add a segmentation label of *shadow*
- Train Pix2pix on the iOCT
- Finetune the pix2pix on OCT

In [ ]:
import os
os.chdir('/home/extra/micheal/IDP')

In [ ]:
from os.path import join, basename
from pathlib import Path
from glob import glob
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
import idp_utils.data_handling.constants as C
from idp_utils.data_handling.ulabel import load_as_array

## Part 1. Dataset Preparation

### 1. Unify aroi labels

Load labels in `split/aroi` dataset, unify the layer labels and remove the fluids


#### Unify and copy labels

In [ ]:
aroi_root = C.SPLIT_PATTERN.format(data='aroi')
unified_aroi_root = C.SPLIT_PATTERN.format(data='uaroi')

In [ ]:
path = aroi_root + "labels/test/patient10_raw0043.png"
arr = load_as_array(path, label_type='aroi')

In [ ]:
def transform_labels(src_root, dst_root, img_types, splits, label_type, save_extension=None):
    """
    Args:
    img_types: ['bscans', 'labels']
    splits: ['train', 'val', 'test']
    save_extension: if left None, the extension will be left untouched
    """
    for typ in img_types:
        for split in splits:
            data_dir = join(src_root, typ, split)
            dst_dir = join(dst_root, typ, split)
            Path(dst_dir).mkdir(parents=True, exist_ok=True)
            img_paths = glob(join(data_dir, '*'))
            for img_path in tqdm(img_paths, desc=typ + ' ' + split):
                img_name = basename(img_path)
                if save_extension is not None:
                    # Remove original extension and append new one
                    img_name = '.'.join(img_name.split('.')[:-1] + [save_extension])
                arg_label_type = label_type if typ == 'labels' else None
                img_array = load_as_array(img_path, label_type=arg_label_type)
                img_processed = Image.fromarray(img_array)
                img_processed.save(join(dst_dir, img_name))

In [ ]:
transform_labels(src_root=aroi_root,
                dst_root=unified_aroi_root,
                img_types=['labels'],
                splits=['train', 'val', 'test'],
                label_type='aroi',
                save_extension='png')

#### Hard link bscans

In [ ]:
# !rm -rf $unified_aroi_root/bscans

In [ ]:
!cp -lR $aroi_root/bscans $unified_aroi_root/bscans

### 2. Unify OP (iOCT) labels

In [ ]:
op_root = C.SPLIT_PATTERN.format(data='ioct')
unified_op_root = C.SPLIT_PATTERN.format(data='uop')

In [ ]:
# unify labels
transform_labels(src_root=op_root,
                dst_root=unified_op_root,
                img_types=['labels'],
                splits=['train', 'val', 'test'],
                label_type='op',
                save_extension='png')

In [ ]:
# hard link bscans
# !cp -lR $op_root/bscans $unified_op_root/bscans

In [ ]:
# verify success
!ls $op_root/bscans

### 3. Create iOCT(op) and OCT(aroi) datasets

`python datasets/combine_A_and_B.py --fold_A /path/to/data/A --fold_B /path/to/data/B --fold_AB /path/to/data`

In [ ]:
uaroi_dataset_dir = C.DATASET_PATTERN.format(data='uaroi')
uop_dataset_dir = C.DATASET_PATTERN.format(data='uop')

In [ ]:
!mkdir -p $uop_dataset_dir $uaroi_dataset_dir

In [ ]:
aroi_label_dir = join(aroi_root, 'labels')
aroi_bscan_dir = join(aroi_root, 'bscans')
op_label_dir = join(op_root, 'labels')
op_bscan_dir = join(op_root, 'bscans')

In [ ]:
# prepare unified AROI dataset
!python pytorch-CycleGAN-and-pix2pix/datasets/combine_A_and_B.py \
    --fold_A $aroi_label_dir \
    --fold_B $aroi_bscan_dir \
    --fold_AB $uaroi_dataset_dir

In [ ]:
# prepare unified OP dataset
!python pytorch-CycleGAN-and-pix2pix/datasets/combine_A_and_B.py \
    --fold_A $op_label_dir \
    --fold_B $op_bscan_dir \
    --fold_AB $uop_dataset_dir

## Part 2. Train a pix2pix on iOCT first

In [ ]:
!python pytorch-CycleGAN-and-pix2pix/train.py --dataroot $uop_dataset_dir \
    --name "uop_pix" \
    --direction AtoB \
    --n_epochs 100 \
    --print_freq 500 \
    --batch_size 64 

Find the location of the artifacts: model weight path

## Part 3. Fine tune on the OCT dataset

TODOs:
1. load weight of the pix2pix trained on the iOCT dataset
2. train the pix2pix on the OCT dataset, but with smaller epoch

Finetune tips: [link](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix/blob/master/docs/tips.md#fine-tuningresume-training)

To fine-tune a pre-trained model, or resume the previous training, use the `--continue_train` flag. The program will then load the model based on `epoch`. By default, the program will initialize the epoch count as 1. Set `--epoch_count <int>` to specify a different starting epoch count.

In [ ]:
!python pytorch-CycleGAN-and-pix2pix/train.py --dataroot $uaroi_dataset_dir \
    --name "uop_pix" \
    --direction AtoB \
    --n_epochs 110 \
    --print_freq 500 \
    --batch_size 64 \
    --continue_train \
    --epoch_count 100